In [ ]:
%load_ext autoreload

In [ ]:
# CONFIG (leave only one uncommented)
config = { # reprocessed officer id
    "grouping_keys": ['driver_first_name', 'driver_last_name', 'DOB'],
    "descript": "_mod_officer_id",
    "base_path": 'replace-with-path-to-this-directory',
    "csv_name": 'path-to-CO-data.csv',
    "hispanic_white_drivers_only_csv_name": 'co_hispanic_white_drivers_only_mod.csv'
}

In [ ]:
# Set base path directory
base_path = config['base_path']

%cd $base_path

!pwd

In [ ]:
%autoreload 2
import pandas as pd
import math
import statistics
import numpy as np
from matplotlib import pyplot as plt
from collections import Counter
from IPython.display import display

from policing_data_expl import *

# Verify Raw Data and Clean Data Match

In [ ]:
filepath = config['csv_name']
dtypes_dict = {k:str for k in config['grouping_keys']}
co_data = standardize_cols('CO', pd.read_csv(filepath, dtype=dtypes_dict))

print(f'Rows: {len(co_data)}')

# Checking Officer ID (Modified now to be the officer first name + last name)

In [ ]:
co_data['officer_id']

In [ ]:
display(co_data[['officer_id', 'officer_first_name', 'officer_last_name']])
officer_info = co_data[['officer_id', 'officer_first_name', 'officer_last_name']]
display(officer_info.groupby(['officer_last_name', 'officer_first_name']).size().reset_index(name='count').sort_values(by='count', ascending=False))

officer_info[(officer_info['officer_first_name'] == 'Scott') & (officer_info['officer_last_name'] == 'Fenwick')].value_counts()

In [ ]:
# 'completeness' (how many non-nan values there are) per column
for column in co_data.columns:
    print(column)
    print('  ', get_percent_complete_column(co_data, column))

In [ ]:
calc_complete_cols(co_data, config['grouping_keys'])

# Construct Filtered Dataset

In [ ]:
grouped_co = group_df_by(co_data, config['grouping_keys'], csv_filename='co_raw_with_driver_id' + config['descript'] + '.csv')

def co_cond(name, entries):
    """
    Only keep drivers 
    - with at least 2 entries (=at least 2 stops but no more than 10 stops)
    - non-null + custom logic for driver_first_name, driver_last_name, DOB (=valid unique identifying features)
    """
    f, l, dob = name
    return len(entries) >= 2 and len(entries) <= 10 and \
        (l != "NOT OBTAINED" and l != "--" and len(f) > 1 and len(l) > 1)
        
csv_name = 'co_grouped' + config['descript'] + '.csv'
check_cond(grouped_co, co_cond, csv_name)

In [ ]:
csv_name = 'co_grouped' + config['descript'] + '.csv'
cogrouped_csv = pd.read_csv(csv_name)

co_grouped = cogrouped_csv.groupby(config['grouping_keys'])

print("#rows of individuals stopped more than once:", len(cogrouped_csv))
print("#individuals stopped more than once:", len(co_grouped))

calc_mean_med_max_stops(co_grouped)

In [ ]:
calc_racial_ambig(co_grouped)

enumerate_racial_ambig(co_grouped)

In [ ]:
cogrouped_csv.head(50)

# Calculate Stats for Racially Ambiguous Subset

In [ ]:
person_race_dict = generate_person_race_dict(co_grouped)
# make the grouping_keys into a tuple so it can be used as a key per person in person_race_dict
tuple_lst = [tuple(keys) for keys in cogrouped_csv[config['grouping_keys']].values.tolist()]
race_str_col = [person_race_dict[(keys)] for keys in tuple_lst]

# call this new column race_str
cogrouped_with_race_str = cogrouped_csv.copy()
cogrouped_with_race_str.insert(2, "race_str", race_str_col, False)

In [ ]:
stats_dict_lst = get_state_stats(cogrouped_csv, race_str_col, config['grouping_keys'])

plot_state_stats(stats_dict_lst, 'CO')

In [ ]:
# remove drivers with more than 10 stops
co_with_drivers_less_than_10_stops = cogrouped_csv.groupby(config['grouping_keys']).filter(lambda x: len(x) <= 10).reset_index()

# make the grouping_keys into a tuple so it can be used as a key per person in person_race_dict
tuple_lst = [tuple(keys) for keys in co_with_drivers_less_than_10_stops[config['grouping_keys']].values.tolist()]
race_str_col = [person_race_dict[(keys)] for keys in tuple_lst]

# call this new column race_str
# azgrouped_with_race_str = azgrouped_csv.copy()
# azgrouped_with_race_str.insert(2, "race_str", race_str_col, False)

stats_dict_lst = get_state_stats(co_with_drivers_less_than_10_stops, race_str_col, config['grouping_keys'])

plot_state_stats(stats_dict_lst, 'CO - ' + config['descript'])

In [ ]:
print(ttest_unpaired(cogrouped_with_race_str))

In [ ]:
print(ttest_paired(co_grouped))

# white-Hispanic Drivers and Regressions

In [ ]:
race_str_cond = cogrouped_with_race_str['race_str'].map(lambda x:x in {"Hispanic_White"})
hispanic_white_drivers = cogrouped_with_race_str.loc[race_str_cond]

print(len(hispanic_white_drivers))
print(config['hispanic_white_drivers_only_csv_name'])
write_to_csv(hispanic_white_drivers, config['hispanic_white_drivers_only_csv_name'])

In [ ]:
cogrouped_with_race_str = pd.read_csv(config['hispanic_white_drivers_only_csv_name'])
cogrouped_with_race_str['officer_id'].value_counts()

In [ ]:
co_hispanic_white_grouped = cogrouped_with_race_str.groupby(config['grouping_keys'])
print(f'entries: {len(cogrouped_with_race_str)}')
print(f'individuals: {len(co_hispanic_white_grouped)}')
calc_mean_med_max_stops(co_hispanic_white_grouped)

In [ ]:
res1 = regress(cogrouped_with_race_str, dep_var='search_conducted', cols=[], controls=[], model_name='No controls', useFixedEffects=True)
res2 = regress(cogrouped_with_race_str, dep_var='search_conducted', cols=['hour_of_day'], controls=['hour_of_day'], model_name='Control for hour of day (linear)', useFixedEffects=True)
res3 = regress(cogrouped_with_race_str, dep_var='search_conducted', cols=['hour_of_day'], controls=['hour_of_day', 'I(hour_of_day**2)', 'I(hour_of_day**3)', 'I(hour_of_day**4)'], model_name='Control for hour of day (quartic)', useFixedEffects=True)
res4 = regress(cogrouped_with_race_str, dep_var='search_conducted', cols=['county_fips'], controls=['C(county_fips)'], model_name='Control for county', useFixedEffects=True)

make_sensitivity_dot_plot([res1, res2, res3, res4], coef_to_plot = 'Hispanic', title='Sensitivity to controls in Colorado')

In [ ]:
# Regressions that don't yield much

# officer_id: there are over 90K officer ids
# res5 = regress(cogrouped_with_race_str, dep_var='search_conducted', cols=['officer_id'], controls=['C(officer_id)'], model_name='Control for officer id - drop absorbed', useFixedEffects=True, drop_absorbed=True)


# Years of Data in the Analysis

In [ ]:
co_complete = pd.read_csv('co_grouped' + config['descript'] + '.csv')

In [ ]:
co_complete['IncidentDate'].map(lambda x: x[:4]).value_counts().sort_index()

In [ ]:
co_hispanic_white = pd.read_csv(config['hispanic_white_drivers_only_csv_name'])

In [ ]:
co_hispanic_white['IncidentDate'].map(lambda x: x[:4]).value_counts().sort_index()

# Comparing a Couple Subsets of the Population

In [ ]:
plot_search_rates_comparison('Colorado', 'search_conducted', co_data, cogrouped_csv, cogrouped_with_race_str)
plot_search_rates_comparison('Colorado', 'is_arrested', co_data, cogrouped_csv, cogrouped_with_race_str)

## All Drivers, white or Hispanic

In [ ]:
plot_top_5_col_values(co_data, 'violation')
plot_top_5_col_values(co_data, 'county_name')

## Multiply Stopped Drivers, white or Hispanic

In [ ]:
plot_top_5_col_values(cogrouped_csv, 'violation')
plot_top_5_col_values(cogrouped_csv, 'county_name')

## Multiply Stopped Drivers with white/Hispanic Racial Ambiguity

In [ ]:
plot_top_5_col_values(cogrouped_with_race_str, 'violation')
plot_top_5_col_values(cogrouped_with_race_str, 'county_name')